<a href="https://colab.research.google.com/github/Danish-Hussain-DSAI/facial-expression-recognition-project-/blob/main/facial_expression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!7za -y x "/content/drive/MyDrive/facial expression project/data/image/origin.7z.*"


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/facial expression project/data/image/                                                                      8 files, 8113576419 bytes (7738 MiB)
  0%    
Extracting archive: /content/drive/MyDrive/facial expression project/data/image/origin.7z.001
--
Path = /content/drive/MyDrive/facial expression project/data/image/origin.7z.001
Type = Split
Physical Size = 1048576000
Volumes = 8
Total Physical Size = 8113576419
----
Path = origin.7z
Size = 8113576419
--
Path = origin.7z
Type = 7z
Physical Size = 8113576419
Headers Size = 863607
Method = LZMA:25
Solid = +
Blocks = 2

  0%  

In [1]:
targets=r"/content/drive/MyDrive/facial expression project/data/label/label.lst"
input_images=r"/content/drive/MyDrive/facial expression project/origin"

In [2]:
import pandas as pd
targets_table =pd.read_csv(targets, sep=" ",header=None)
table_cols="image_name face_id_in_image face_box_top face_box_left face_box_right face_box_bottom face_box_cofidence expression_label".split(" ")
targets_table.columns=table_cols
targets_table.head()

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0


In [3]:
filtered_images=targets_table[targets_table.face_box_cofidence>40]

In [4]:
filtered_images.shape

(57931, 8)

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd

def process_images_in_batches(data_frame, data_directory, batch_size=32, target_size=(64, 64)):
    if "image_name" not in data_frame.columns:
        raise KeyError("DataFrame does not contain 'image_name' column.")

    num_samples = len(data_frame)
    batches = (num_samples + batch_size - 1) // batch_size

    images_set = []
    labels_set = []

    for i in range(batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, num_samples)
        batch_data = data_frame.iloc[start_idx:end_idx]

        batch_images = []
        batch_labels = []
        for _, row in batch_data.iterrows():
            img_name = row["image_name"]
            img_path = os.path.join(data_directory, img_name)

            # Load the image
            filtered_image = cv2.imread(img_path)
            if filtered_image is not None:
                x1 = row["face_box_left"]
                x2 = row["face_box_right"]
                y1 = row["face_box_top"]
                y2 = row["face_box_bottom"]
                targets = row["expression_label"]

                cropped_image = filtered_image[y1:y2, x1:x2]
                if cropped_image is not None:
                    cropped_resized = cv2.resize(cropped_image, target_size)
                    batch_images.append(cropped_resized)
                    batch_labels.append(targets)

        batch_images = np.array(batch_images)
        batch_labels = np.array(batch_labels)

        images_set.append(batch_images)
        labels_set.append(batch_labels)

    return np.concatenate(images_set), np.concatenate(labels_set)

# Example usage:
data_directory = input_images  # Directory containing the images
filtered_images = filtered_images  # Your DataFrame with 'image_name', 'face_box_left', 'face_box_right', 'face_box_top', 'face_box_bottom', and 'expression_label'

batch_size = 32
target_size = (64, 64)

# Process images in batches
images_set, labels_set = process_images_in_batches(filtered_images, data_directory, batch_size=batch_size, target_size=target_size)

# Now you have the cropped images in 'images_set' and corresponding labels in 'labels_set'


In [ ]:
import os
import numpy as np
import cv2

In [ ]:

images_set=[]
labels_set=[]
for i, row in filtered_images.sample(100).iterrows():
    img_name = row["image_name"]
    x1 = row["face_box_left"]
    x2 = row["face_box_right"]
    y1 = row["face_box_top"]
    y2 = row["face_box_bottom"]
    targets=row["expression_label"]
    img_path =os.path.join(input_images, img_name)


    # Load the image
    filtered_image = cv2.imread(img_path)

    if filtered_image is not None:
       cropped_image=filtered_image[y1:y2,x1:x2]
    else:
       continue

    if cropped_image is not None:

      cropped_resized=cv2.resize(cropped_image,(64, 64))

    else:
     continue

    images_set.append(cropped_resized)
    labels_set.append(targets)












In [ ]:
print(len(images_set))
print(len(labels_set))

83
83


In [ ]:
images_numpy=np.array(images_set)
labels_numpy=np.array(labels_set)



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

# Assuming you have x_numpy and y_numpy

# Normalize the image data
x_normalized = images_numpy.astype('float32') / 255.0

# Apply SMOTE to balance the dataset
ros = RandomOverSampler(random_state=42)
x_resampled, y_resampled = ros.fit_resample(x_normalized.reshape(-1, 64*64*3), labels_numpy)

# Split the resampled data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=42)

# Reshape x_train and x_test to match the input shape of the model
x_train = x_train.reshape(-1, 64, 64, 3)
x_test = x_test.reshape(-1, 64, 64, 3)

# Data augmentation using ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)
datagen.fit(x_train)

num_classes = len(np.unique(y_train))

# Load pre-trained VGG16 model (excluding the top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Multi-class classification

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use sparse categorical cross-entropy for integer labels
              metrics=['accuracy'])

# Fit the model on the training data with data augmentation
model.fit(datagen.flow(x_train, y_train, batch_size=32), steps_per_epoch=len(x_train) // 32, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.2f}')


Epoch 1/10
864/864 [==============================] - 52s 59ms/step - loss: 1.7865 - accuracy: 0.2874 - val_loss: 1.6667 - val_accuracy: 0.3562
Epoch 2/10
864/864 [==============================] - 49s 57ms/step - loss: 1.6875 - accuracy: 0.3399 - val_loss: 1.6071 - val_accuracy: 0.3980
Epoch 3/10
864/864 [==============================] - 47s 55ms/step - loss: 1.6390 - accuracy: 0.3673 - val_loss: 1.5262 - val_accuracy: 0.4254
Epoch 4/10
864/864 [==============================] - 50s 57ms/step - loss: 1.6004 - accuracy: 0.3799 - val_loss: 1.5207 - val_accuracy: 0.4346
Epoch 5/10
864/864 [==============================] - 47s 55ms/step - loss: 1.5698 - accuracy: 0.3975 - val_loss: 1.4635 - val_accuracy: 0.4624
Epoch 6/10
864/864 [==============================] - 47s 55ms/step - loss: 1.5450 - accuracy: 0.4070 - val_loss: 1.4330 - val_accuracy: 0.4751
Epoch 7/10
864/864 [==============================] - 49s 57ms/step - loss: 1.5307 - accuracy: 0.4144 - val_loss: 1.4555 - val_accuracy: